# Wiki toru (3) tutorial Python notebook

Use this notebook to help you code and calculate answers to the Week 3 tutorial questions. We'll assume you're familar with Python operations now, so we'll skip all our coding reminders.

You can refresh your memory of useful equations from your notes, or from the lecture summary notes on Learn.

I've loaded some potentially useful packages for the tutorial in the first block.


In [1]:
# Load packages - add any others you might need
import pandas as pd
import numpy as np
import math

In [2]:
# Begin code! 
# Question 1
# define function for power
def power(H,Q,eta):
    #H is head in m
    #Q is flow in m3/s
    #eta is efficiency
    P = 1000*9.81*H*Q*eta
    return P

H = 50 #m
Q = 80 #m3/s

#1a) calculate the power in the water
P_1a = power(H,Q,1)/1000000 #convert to MW
print('The power in the water is {power:.2f} MW'.format(power=P_1a))

#1b) calculate max capacity of each turbine, no losses
P_1b = P_1a/4 #or = power(H,Q/4,1)/1000000
print('The maximum capacity for each turbine is {power:.2f} MW'.format(power=P_1b))

#1d) how much electricity in a year?
HL = 0.1
eta_tg = 0.93
CF = 0.72

eta_T = (1-HL)*eta_tg

P_1d = power(H,Q,eta_T)/1000000 #in MW OR power((H-HL),Q,eta_tg)
Gen_1d = P_1d * 8760 * CF

print('The expected annual generation is {energy:.0f} GWh'.format(energy=Gen_1d/1000))

The power in the water is 39.24 MW
The maximum capacity for each turbine is 9.81 MW
The expected annual generation is 207 GWh


In [3]:
# Question 2
# define function for runoff flow
def Q_r(i,A,c):
    #i is rainfall in m3/m2/s
    #A is area of catchment
    #c is runoff coefficient
    runoff = i*A*c
    return runoff

# known values
rainfall = 800/1000/365/24/60/60 #800 mm/yr converted to m3/m2/s
A_c = 22*1000000 #22km2 converted to m2
slope = 0.11
c = 0.1
L_p = 1000 #penstock length in m

# calculate the head at the site
H_gross = L_p*slope

# calculate power in the water
runoff = Q_r(rainfall,A_c,c)
P_2a = power(H_gross,runoff,1)
print('The power in the water is {power:.2f} kW'.format(power=P_2a/1000))

# Calculate head loss
f = 0.013
D = 0.2

v = runoff/(np.pi*(D/2)**2)
HL = (f*L_p*v**2)/(9.81*2*D)
H_eff = H_gross - HL

# calculate estimated electricity gen
eta_t = 0.87
eta_g = 0.9
eta_tot = eta_t*eta_g

Gen_2b = power(H_eff,runoff,eta_tot)*8760
print('The expected annual generation is {energy:.0f} MWh'.format(energy=Gen_2b/1000000))

The power in the water is 60.22 kW
The expected annual generation is 374 MWh


In [4]:
# Challenge
# new known values
eta_gen = 0.9

# set up lists for calculations in each month
months = list(np.arange(1,13,1))
days = [31,28,31,30,31,30,31,31,30,31,30,31]
rainfall = []
flow = []
headloss = []
head_eff = []
P_available = []
eta_turbine = []
P_gen = []
Generation = []

rainfall = []
for j in range(0,len(months)):
    #calculate rainfall for each month with sine function
    r = 62.5*math.sin((months[j]+8)*np.pi/6)+67.5
    rainfall.append(r)

    #convert rainfall values to m3/m2/s
    i = rainfall[j]/1000/days[j]/24/60/60

    #calculate flow and velocity
    Q_rain = i*A_c*c
    v_rain = Q_rain/(np.pi*(D/2)**2)

    #calculate head loss and effective head
    H_loss = (f*L_p*v_rain**2)/(2*9.81*D)
    H_eff = H_gross - H_loss

    #calculate turbine efficiency based on flow
    if Q_rain < 0.011:
            eta_turb = 0
    else:
        if Q_rain > 0.095:
            eta_turb = 0.87
        else: 
            a = 0.87/(0.095-0.011)
            b = 0.87-a*0.095
            eta_turb = a*Q_rain + b
    eta_turbine.append(eta_turb)

    #calculate total energy generated in a month
    eta_total = eta_turb*eta_gen
    E = power(H_eff,Q_rain,eta_total)*days[j]*24
    Generation.append(E) 

#sum monthly generation
total_gen = np.sum(Generation)/1000000 #convert to MWh
print('The expected annual generation is {energy:.0f} MWh'.format(energy=total_gen))


The expected annual generation is 241 MWh
